In [ ]:
###################### DUE TO LACK OF COMPUTATION POWER I CANNONT TRAIN THIS MODEL ######################

# may god bless the person who runs it on their pc

# link to the dataset
# https://www.kaggle.com/c/cifar-10/data

"""
    Image classification using convolutional neural network
    dataset: CIFAR-10
    Framework: Tensorflow
"""

import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import cv2
from IPython.display import clear_output

In [ ]:
# creating one-hot matrix of the output labels
# base is the set of training image labels

# compares the label in base with the label in label_list
# 1 is stored in the one_hot matrix at
# the index at which the label from base lies in the label_list 

def one_hot(base):
    label_list = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    # one_hot matrix
    hot_matrix = np.zeros([np.size(base[:, 0]), 10], dtype = float)
    for i in range(np.size(base[:, 0])):
        hot_matrix[i, label_list.index(base[i, 1])] = 1
    return hot_matrix

In [ ]:
# creating minibatches
def mini_batch(X, Y, size):
    idx = np.random.randint(np.size(Y[:, 0]), size = (size,1))
    x_bat = X[idx]
    x_bat = x_bat.reshape(size, 32, 32, 3)
    y_bat = Y[idx]
    y_bat = y_bat.reshape(size, 10)
    
    return x_bat, y_bat

In [ ]:
# importing 50000 images of size 32x32x3
i = 0
img_base = []
for img in glob.glob("train\\*.png"):
    img_base.append(cv2.imread(img))
        
print('imported ', i, 'images!')
x_train = np.array(img_base[0:40000]).astype(np.float32)

In [ ]:
# importing labels
labels = pd.DataFrame(pd.read_csv("trainLabels.csv"))
y_train = np.array(labels[0:40000])
y_train = one_hot(y_train)
print('training labels size: ',y_train.shape)
print('training images size: ', x_train.shape)

In [ ]:
# to reset the graph
tf.reset_default_graph()

# creating placeholders
x = tf.placeholder(tf.float32, [None, 32, 32, 3])
y = tf.placeholder(tf.float32, [None, 10])

# creating weights
w1 = tf.get_variable('w1', [4,4,3,10], initializer=tf.contrib.layers.xavier_initializer())
w2 = tf.get_variable('w2', [4,4,10,15], initializer=tf.contrib.layers.xavier_initializer())

In [ ]:
# forward propagation
with tf.device("/gpu:0"):
    # convolution layer 1
    c1 = tf.nn.conv2d(x_train, w1, strides = [1,1,1,1], padding = 'VALID')
    # activation function for c1: relu
    r1 = tf.nn.relu(c1)
    # maxpooling
    p1 = tf.nn.max_pool(r1, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

    # convolution layer 2
    c2 = tf.nn.conv2d(p1, w2, strides = [1,1,1,1], padding='VALID')
    # activation function for c2: relu
    r2 = tf.nn.relu(c2)
    # maxpooling
    p2 = tf.nn.max_pool(r2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

    # flattening the previous max pool layer
    l1 = tf.contrib.layers.flatten(p2)
    # fully connected layer
    final = tf.contrib.layers.fully_connected(l1, 10, activation_fn = None)

In [ ]:
# hyperparameters
learning_rate = 0.05

# epochs = no. of training iterations 
epochs = 1000

In [ ]:
# training and optimization

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = final, labels = y_train))
# using adam optimizer 
optimize = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

In [ ]:
# creating tensorflow session
se = tf.Session(config=tf.ConfigProto(log_device_placement=True))

# initializing variables
se.run(tf.global_variables_initializer())

# training the graph
# mini-batch size = 100
for i in range(epochs):
    x_batch, y_batch = mini_batch(x_train, y_train, 100)
    training_step = se.run(optimize, {x: x_batch, y: y_batch})
    
cost = se.run(cross_entropy, {x: x_train, y: y_train})
print('cost: ', cost)

In [ ]:
# metrics

# importing test set
x_test = np.array(img_base[40000:]).astype(np.float32)
y_test = np.array(labels[40000:])
y_test = one_hot(y_test)

# training set accuracy
# 1. predictions
train_predictions = se.run(final, {x: x_train, y:y_train})
# 2. correct results
correct_train_predictions = tf.equal(tf.argmax(predictions, 1), tf.argmax(y_train, 1)) 
training_accuracy = tf.reduce_mean(tf.cast(correct_train_predictions, dtype = 'float'))

# test set accuracy
# 1. predictions
test_predictions = se.run(final, {x: x_test, y:y_test})
# 2. correct results
correct_test_predictions = tf.equal(tf.argmax(predictions, 1), tf.argmax(y_test, 1)) 
test_accuracy = tf.reduce_mean(tf.cast(correct_test_predictions, dtype = 'float'))

print('training accuracy: ', se.run(train_accuracy))
print('training accuracy: ', se.run(test_accuracy))